# PART 3: Connect with Manufacturers for remote SMC

In [1]:
%%javascript
document.title="City Agent"

<IPython.core.display.Javascript object>

#### Imports

In [2]:
%autoawait 
from aries_cloudcontroller import AriesAgentController
#import asyncio
import os
from pprintpp import pprint
from termcolor import colored
import time

IPython autoawait is `on`, and set to use `asyncio`


In [3]:
# Global connection ID to enable the use of an multi-use invite to M1, M2, M3
# This way, CONNECTION_ID is updated depending on who connects with the City agent
global CONNECTION_ID

In [4]:
# If the identifiers are not stored for some reason
try:
    # Load variables – assuming they have been written to the store in a previous notebook
    print("--- LOAD VARIABLES FROM JUPYTERLAB STORE ---")
    
    %store -r schema_manufacturer_id
    %store -r cred_def_manufacturer_id
    %store -r authority_did
    
    print(f"schema_manufacturer_id: {schema_manufacturer_id}")
    print(f"cred_def_manufacturer_id: {cred_def_manufacturer_id}")
    print(f"authority_did: {authority_did}")
    
except Exception as e:
    
    print("--- DEFINE VARIABLES FROM MANUALLY COPIED identifiers DICT ---")
    
    # Copy an updated version if necessary
    identifiers = {
        'authority_did': 'MTXSc4YD8wixyM9ekZbmMC',
        'city_schema_identifiers': {
            'cred_def': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246685:default',
            'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-city-agency:0.0.1',
        },
        'manufacturer_schema_identifiers': {
            'cred_def': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246687:default',
            'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1',
        },
    }

    # Get manufacturer schema identifiers
    schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
    cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]
    
    # Get authority did
    authority_did = identifiers['authority_did']

    # Load variables to Jupyter store
    %store schema_manufacturer_id
    %store cred_def_manufacturer_id
    %store authority_did

--- LOAD VARIABLES FROM JUPYTERLAB STORE ---
schema_manufacturer_id: MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1
cred_def_manufacturer_id: MTXSc4YD8wixyM9ekZbmMC:3:CL:246687:default
authority_did: MTXSc4YD8wixyM9ekZbmMC


#### Initialize `City` Agent Controller

In [5]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [6]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [7]:
# Receive connection messages
def connections_handler(payload):
    global CONNECTION_ID
    
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    rfc_state = payload["rfc23_state"]
    
    CONNECTION_ID = connection_id
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : {routing} ({rfc})".format(routing=routing_state, rfc=rfc_state))
    if 'their_label' in payload: 
        print(f"Connection with : ", payload['their_label'])
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
        
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
        
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

        
def verifier_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    

    if state == "request_sent":
        print("Presentation Request\n")
        print(payload["presentation_request"])
        print("\nThe presentation request is encoded in base64 and packaged into a DIDComm Message\n")
        print(payload["presentation_request_dict"])
        print("\nNote the type defines the protocol present-proof and the message request-presentation\n")
        print(payload["presentation_request_dict"]["@type"])
    elif state == "presentation_received":
        print("Presentation Received")
        print("We will not go into detail on this payload as it is comparable to the presentation_sent we looked at in the earlier cell.")
        print("This is the full payload\n")
        print(payload)
    else:
        print("Paload \n")
        print(payload)

In [8]:
# Init listener list
listeners = []

# Add listeners defined in previous cell
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

verifier_listener = {"topic": "present_proof", "handler": verifier_proof_handler}
listeners.append(verifier_listener)

# Register listeners
agent_controller.register_listeners(listeners)

## Define Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections requesting that they produce a presentation containing the identified attributes and meeting the restrictions.

It is often useful to define your request objects first, then reuse these objects across many connections you wish to request a proof from. 

Duplicate and customise the below cell as many times as you need. It may be useful to save these request objects either to the jupyter store using %store or through

TODO: Detail the full set of restrictions available to a verifier.

In [9]:
# We add a constraint that the attribute must originate from this schema

# Define the list of attributes and restrictions under which each attribute was issued that a prover must satisfy with a presentation
# NOTE: if identifying a schema or credential definition then the attribute name must be contained within the corresponding schema.
req_attrs = [
    {"name": "isManufacturer", "restrictions": [{"schema_id": schema_manufacturer_id}]},
]

# We could extend this to request the name attribute aswell if we wanted.


manufacturer_proof_request = {
    "name": "isManufacturer Proof Request",
    "version": "1.0",
    "requested_attributes": {
        # They generally follow this uuid pattern. Unique identifier for attribute within context of this proof request
        # Note that req_attr['name'] gets the attribute name of each object. E.g. domain and name in this case
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
    # You can also request the entire proof request be non-revoked
    "non_revoked":  {"to": int(time.time())}
}

In [10]:
manufacturer_proof_request

{'name': 'isManufacturer Proof Request',
 'version': '1.0',
 'requested_attributes': {'0_isManufacturer_uuid': {'name': 'isManufacturer',
   'restrictions': [{'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1'}]}},
 'requested_predicates': {},
 'non_revoked': {'to': 1628520443}}

## Create Invitation
Note the current arguments specified are in their default configurations. 

In [11]:
alias = None # Alias for invited connection
auto_accept = "true" # Auto accept response
public = "false" # Don't use a public DID
multi_use = "true" # Use invitation for multiple invitees

invite = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  4ebc01fc-d06b-44b6-9aa2-f865c4df25ff
State :  invitation
Routing State : none (invitation-sent)
Their Role :  invitee
----------------------------------------------------------
invitation


## Share Invitation Object with External Agent

Typically in this jupyter notebook playground that involves copying it across to another agent's business logic notebook where they are the invitee. (see prover_template)

In [12]:
invitation = invite["invitation"]
pprint(invitation) 

{
    '@id': '4b978b6d-8381-406e-bbe8-c7182ebe0dc8',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['842s8grD4E1x59xDyS3dnF8YwMqRFMXML5zW2KvakbZb'],
    'serviceEndpoint': 'https://80a67f24c0cb.ngrok.io',
}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f6824b3d-c994-4915-86eb-7e653a74ee1f
State :  invitation
Routing State : none (invitation-sent)
Their Role :  invitee
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f6824b3d-c994-4915-86eb-7e653a74ee1f
State :  request
Routing State : none (request-received)
Connection with :  Manufacturer1
Their Role :  invitee
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
C

## Optional: Recieve Proof Proposal

A Verifier is not the only role that can initiate the present-proof protocol. A holder can also send a proposal for a presentation that they can provide.

In [ ]:
# TODO...

## Send Proof Request

This identifies a connection and a proof request object (you will need to update the variable name enclosed with <>) 

In [13]:
# Step 1

proof_request = {
    "comment": "Please prove that you are an agent who is a certified manufacturer",
    "connection_id": CONNECTION_ID,
    "proof_request": manufacturer_proof_request,
    # Do you want your agent to trace this request (for debugging)
    "trace": True
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  f6824b3d-c994-4915-86eb-7e653a74ee1f
Presentation Exchange ID :  3b96d36b-5192-455e-8768-a387b5310608
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'nonce': '544370537991510495111363', 'name': 'isManufacturer Proof Request', 'version': '1.0', 'requested_attributes': {'0_isManufacturer_uuid': {'restrictions': [{'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1'}], 'name': 'isManufacturer'}}, 'requested_predicates': {}, 'non_revoked': {'to': 1628520443}}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'e28a0f01-0b8e-49bb-983d-157f00b485aa', '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comme

*I THINK* **Break point**: see step 2

---

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [14]:
presentation_exchange_id = proof_request_response["presentation_exchange_id"]
pres_record = await agent_controller.proofs.get_record_by_id(presentation_exchange_id)
print(pres_record)

{'connection_id': 'f6824b3d-c994-4915-86eb-7e653a74ee1f', 'presentation_exchange_id': '3b96d36b-5192-455e-8768-a387b5310608', 'thread_id': 'e28a0f01-0b8e-49bb-983d-157f00b485aa', 'auto_present': False, 'initiator': 'self', 'trace': True, 'presentation_request': {'nonce': '544370537991510495111363', 'name': 'isManufacturer Proof Request', 'version': '1.0', 'requested_attributes': {'0_isManufacturer_uuid': {'restrictions': [{'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1'}], 'name': 'isManufacturer'}}, 'requested_predicates': {}, 'non_revoked': {'to': 1628520443}}, 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'ismanufacturer': '101276561079705913907459647512048005855328137776566494081283796781441440993409'}, 'a_prime': '68538606484741577597416790060018185640495006653054521454444353732998479644253611931462894653790548698618132960073516558580459479631470958339407955801935526629754613363001222677308429477363359069194124913174996

## Verify Presentation

Only if it is in the right state. 

Note: Verifying a presentation moves the state to `verified` regardless of whether the presentation request has been satisfied. To check this you must refer to the `verified` property on the response.

In [15]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

verified = verified_response["verified"]


---------------------------------------------------------------------

Handle present-proof
Connection ID :  f6824b3d-c994-4915-86eb-7e653a74ee1f
Presentation Exchange ID :  3b96d36b-5192-455e-8768-a387b5310608
Protocol State :  verified
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Paload 

{'connection_id': 'f6824b3d-c994-4915-86eb-7e653a74ee1f', 'presentation_exchange_id': '3b96d36b-5192-455e-8768-a387b5310608', 'thread_id': 'e28a0f01-0b8e-49bb-983d-157f00b485aa', 'auto_present': False, 'initiator': 'self', 'trace': True, 'presentation_request': {'nonce': '544370537991510495111363', 'name': 'isManufacturer Proof Request', 'version': '1.0', 'requested_attributes': {'0_isManufacturer_uuid': {'restrictions': [{'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-manufacturer:0.0.1'}], 'name': 'isManufacturer'}}, 'requested_predicates': {}, 'non_revoked': {'to': 1628520443}}, 'presentation': {'proof': {'proofs': [{'primary_pr

## Parsing Disclosed Attribute Values from Presentation

A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

### Parse Revealed Attributes

In [16]:
for (name, val) in verified_response['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("\nAttribute : ", val)
    
    attr_name = verified_response["presentation_request"]["requested_attributes"][name]["name"]
    print("Attribute Name :  Raw Value")
    print(f"{attr_name}   :  {val['raw']}")


Attribute :  {'raw': 'TRUE', 'sub_proof_index': 0, 'encoded': '101276561079705913907459647512048005855328137776566494081283796781441440993409'}
Attribute Name :  Raw Value
isManufacturer   :  TRUE


### Parse Self-Attested Attributes

In [18]:
for (name, val) in verified_response['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
# Imports
import torch
import syft as sy

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

In [2]:
# Step 1: Create duet
duet_ds = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: c119db5cdf423888d5a631b1f568b720

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("c119db5cdf423888d5a631b1f568b720")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!


♫♫♫ > Duet Partner's Client ID:  ff29b1c96f3d59db8d163ae3a61299ad



♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



In [3]:
# Step 4: Connect to duet of M1
duet_m1 = sy.duet("bf2a7a2d63c30294c9bfa5c61ea9be9e")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 5f521eaa1db551edb939fda2fba098f2

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


## 2 - Secure Multi-Party Computation

In [9]:
# Step 5: Setup session with all duets
session = Session(parties=[duet_ds, duet_m1])
print(session)

In [10]:
# Step 6: Setup MPC session
SessionManager.setup_mpc(session)

In [ ]:
# Step 7: NOW we're ready to set up private operaitions
duet_m1.store.pandas

In [12]:
# Step 12: basic operations
x_secret = duet_m1.store["TEST_#1_from_m1_to_ds"] # describe local data to test sum, substract, and multiply

KeyboardInterrupt: 

In [ ]:
y = torch.Tensor([[-5,8,1,7,6,100]]) # Local data

In [ ]:
x = MPCTensor(secret=x_secret, shape=(1,), session=session)  # MPC Tensor from x_secret

The following lines of codes always receive an error message.
[See Github issue](https://github.com/OpenMined/SyMPC/issues/282).

In [ ]:
print("X + Y = ",(x + y).reconstruct())

In [ ]:
print("X - Y = ", (x - y).reconstruct())

In [ ]:
print("X * Y = ", (x * y).reconstruct())

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()